In [ ]:
import riskparity
import mpt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import numpy as np
import pandas as pd
from utils import *
import matplotlib.pyplot as plt


def optimal_portfolio_returns(tickers, b, year_range_start, years, short_run_years):
    yearly_rf_rate = 0.03

    # Get the current day
    current_date = datetime.strptime(year_range_start, "%Y-%m-%d")

    # Get the start day for the data (>2 years in advance for ex-ante predictions)
    start_date_for_data = (current_date - relativedelta(months = 25)).strftime("%Y-%m-%d")

    # Get the final date for the data
    final_date, final_date_str = year(year_range_start, years)

    # Download the data
    data = yf.download(tickers, start=start_date_for_data, end=final_date_str)["Close"]
    data = data.fillna(method='pad')
    print(data)

    # Download market data for market predictions (1 year in advance)
    market_data = yf.download("^GSPC", start=start_date_for_data, end=final_date_str)["Close"]

    # find total number of trading days 
    trading_days = get_trading_days(year_range_start, final_date_str)

    # Get the daily risk free rate and the overall risk free rate
    daily_rf_rate = (1+yearly_rf_rate)**(1/252)-1 # daily risk free rate
    risk_free_rate_over_lr = (1+yearly_rf_rate)**(years)-1

    # Get the overall MSRP sharpe ratio
    print("=================START=================")
    print("LR RFR:", risk_free_rate_over_lr)
    long_run_mpt_sharpe = mpt.find_best_portfolio(data, market_data, daily_rf_rate, year_range_start, final_date_str)
    print("LONG RUN MPT SHARPE:", long_run_mpt_sharpe)
    print("=================END=================")

    # Initialize overall RP sharpe ratio variables
    overall_rp_returns = []
    overall_rp_vol = 0

    # Initialize arrays for data collection
    # column_names = ['x_axis_mpt']
    # for i in range(0, total_rccs, 1):
    #     macy_interval = i * step
    #     column_name = 'y_axis_rp_' + macy_interval
    #     column_names.append(column_name)
    # df = pd.DataFrame(columns=column_names)
    # print(df)

    x_axis_mpt = []
    y_axis_rp = []

    # Loop through all short run periods, each (short_run_years) years on a 6 month sliding window basis
    counter = 0
    end_sr_date = current_date + relativedelta(years=short_run_years) 
    while (end_sr_date < final_date): 
        print("\n==========================================================")
        print("CURRENT WINDOW:", current_date, " - ", end_sr_date, "\n")

        # Get the optimal MPT Portfolio for the short run
        sharpe = mpt.find_best_portfolio(data, market_data, daily_rf_rate, current_date, end_sr_date)
        x_axis_mpt.append(sharpe)

        # Ex-Ante Risk Parity Portfolio variables
        current_ex_ante_date = current_date
        monthly_returns = []
        total_variance = 0
        # Get new weights every 1 month within (short_run_years) years
        while current_ex_ante_date <= end_sr_date and (current_ex_ante_date + relativedelta(days = 32) < final_date ):
            # Use data from two years prior to make predictions
            _, two_years_prior_str = year(current_ex_ante_date, -2)
            end_ex_ante_date = current_ex_ante_date + relativedelta(months = 1)
            # print("CURRENT EX-ANTE WINDOW:", current_ex_ante_date, " - ", end_ex_ante_date)

            # Get RRC weights using two years prior until now 
            rp_weights = riskparity.produce_rbp_stats(data, b, two_years_prior_str, current_ex_ante_date.strftime("%Y-%m-%d"))

            # Get returns for the month
            month_returns, month_variance =  mpt.portfolio_returns(current_ex_ante_date.strftime("%Y-%m-%d"), end_ex_ante_date.strftime("%Y-%m-%d"), data, rp_weights)
            
            # Increment returns and variance accordingly
            monthly_returns.append(month_returns)
            total_variance += month_variance

            # Move to the next month
            current_ex_ante_date = current_ex_ante_date + relativedelta(months=1)

        # Get sharpe ratio of RP Portfolio over the entire short term
        volatility = np.std(monthly_returns) * (12 * short_run_years)**(0.5) 

        total_returns = np.prod(1 + np.array(monthly_returns)) - 1
        risk_free_rate_st = (yearly_rf_rate + 1)**(short_run_years) - 1             # get the risk free rate in the short run
        rp_sharpe = (total_returns - risk_free_rate_st) / volatility                    # get the portfolio's sharpe ratio
        print(total_returns, "-", risk_free_rate_st, "/", volatility, "=", rp_sharpe) 
        print("MSRP:", sharpe, " | ", "RP:", rp_sharpe) 

        y_axis_rp.append(float(rp_sharpe))

        if (counter % (2 * short_run_years) == 0):
            overall_rp_returns.append(total_returns)
            overall_rp_vol += total_variance

        # Increment the current and end dates
        current_date += relativedelta(months = 6)
        end_sr_date += relativedelta(months = 6)
        counter += 1

    # Get long run sharpe
    # volatility_lr = overall_rp_vol**0.5
    volatility_lr = np.std(overall_rp_returns) * (years / short_run_years)**0.5
    
    overall_rp_return = np.prod(1 + np.array(overall_rp_returns)) - 1 
    long_run_rp_sharpe = (overall_rp_return - risk_free_rate_over_lr) / (volatility_lr)
    print("OVERALL CALCULATION:", overall_rp_return, "-", risk_free_rate_over_lr, "/", volatility_lr)

    return x_axis_mpt, y_axis_rp, float(long_run_mpt_sharpe), float(long_run_rp_sharpe)



def rrc_score(x_axis_mpt, y_axis_rp):
  return np.sum((np.array(y_axis_rp)- np.array(x_axis_mpt))) / len(y_axis_rp)

# tickers = ["^GSPC", "^TNX", "^SPGSCI"]
# b = np.array([1, 1, 1])
# # start_date="1986-01-03"
# # years = 39

# start_date="2000-01-03"
# years = 24


# x_axis_mpt, y_axis_rp, long_run_mpt_sharpe, long_run_rp_sharpe  = optimal_portfolio_returns(tickers, b, start_date, years, 1)
# print("RED DOT:", long_run_mpt_sharpe, long_run_rp_sharpe)
# plt.figure()
# plt.scatter(x_axis_mpt, y_axis_rp)
# x_line = np.linspace(-1, 3, 10)  # Adjust range if needed
# plt.plot(x_line, x_line, color='black', label='y = x')
# plt.plot(long_run_mpt_sharpe, long_run_rp_sharpe, marker='o', color='red')
# plt.xlabel("Ex-Post Sharpe Ratios")
# plt.ylabel("Ex-Ante Sharpe Ratios")
# plt.show()
 


In [ ]:
def get_the_macy_ratio():
  tickers = ["^GSPC", "^TNX", "^SPGSCI"]
  b = np.array([1.0, 1.0, 1.0])

  macy_constants = []
  scores = []
  columns = ['macy_constant', 'score', 'long_run_dif']
  df = pd.DataFrame(columns=columns)
  for i in range(50, 51, 1):
  # for i in range(32, 50, 2):
    b[0] = i / 100
    b[1] = (1 - i/100) / 2 
    b[2] = (1 - i/100) / 2 
    macy_constant = b[0]
    print("B:", b)

    
    start_date="1986-01-03"
    # start_date="2000-01-03"
    # start_date="2022-01-03"
    years = 39

    x_axis_mpt, y_axis_rp, long_run_mpt_sharpe, long_run_rp_sharpe = optimal_portfolio_returns(tickers, b, start_date, years, 5)


  
    plt.figure()
    plt.scatter(x_axis_mpt, y_axis_rp)
    x_line = np.linspace(0, 3, 10)  # Adjust range if needed
    plt.plot(long_run_mpt_sharpe, long_run_rp_sharpe, marker='o', color='red')
    plt.plot(x_line, x_line, color='black', label='y = x')
    plt.xlabel("Ex-Post Sharpe Ratios")
    plt.ylabel("Ex-Ante Sharpe Ratios")
    plt.show()

    score = rrc_score(x_axis_mpt, y_axis_rp)
    print(score)

    long_run_dif = long_run_rp_sharpe - long_run_mpt_sharpe
    if (long_run_dif >= 0.2):
      macy_constants.append(float(macy_constant))
      scores.append(float(score))

    new_row = {'macy_constant': macy_constant, 'score': float(score), 'long_run_dif': long_run_dif}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

  
  print(scores)
  print(macy_constants)
  df.to_excel("results.xlsx")
  return float(macy_constants[np.argmax(scores)])

get_the_macy_ratio()
    


B: [0.5  0.25 0.25]
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed
C:\Users\jmacy\AppData\Local\Temp\ipykernel_18592\1669052139.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='pad')
[*********************100%***********************]  1 of 1 completed


Ticker            ^GSPC     ^SPGSCI    ^TNX
Date                                       
1983-12-05   165.759995         NaN  11.780
1983-12-06   165.470001         NaN  11.760
1983-12-07   165.910004         NaN  11.790
1983-12-08   165.199997         NaN  11.890
1983-12-09   165.080002         NaN  11.900
...                 ...         ...     ...
2024-12-26  6037.589844  540.000000   4.579
2024-12-27  5970.839844  543.150024   4.619
2024-12-30  5906.939941  547.909973   4.545
2024-12-31  5881.629883  549.640015   4.573
2025-01-02  5868.549805  555.630005   4.575

[10353 rows x 3 columns]
=================START=================
LR RFR: 2.1670269825233763
YEARLY EXPECTED MARKET RETURNS 0.006377377416025842
Daily RFR: 0.00011730371383444904  Trading Days: 9829 Overall Risk Free: 2.1673984865489455
=====MSRP: 1.5658099441788376 - 2.1673984865489455 / 1.1985611787204165 = -0.5019256030070686
MAX_SHARPE RETURN: 1.5658099441788376
MAX_SHARPE VOLATILITY: 1.1985611787204165
LONG RUN MPT SHAR